<a href="https://colab.research.google.com/github/biruk50/Medium_articles/blob/main/conways_variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install colorama

In [ ]:
import numpy as np
import time
import os
from colorama import Fore, Style, init
from math import log2
import matplotlib.pyplot as plt

# Initialize colorama for cross-platform colored text support
init(autoreset=True)

class GameOfLife:
    def __init__(self, rows, cols, birth_rules=[3], survival_rules=[2, 3]):
        self.rows = rows
        self.cols = cols
        self.board = np.random.randint(2, size=(self.rows, self.cols))
        self.birth_rules = birth_rules  # Neighbors count for cell to be born
        self.survival_rules = survival_rules  # Neighbors count for cell to survive
        self.previous_population = np.sum(self.board)
        self.neighbor_counts = []  # List to store neighbor counts for each iteration
        self.populations = []  # Track population over iterations
        self.population_changes = []  # Track population change over iterations
        self.entropies = []  # Track entropy over iterations

    def count_neighbors(self, row, col):
        total = 0
        for i in range(-1, 2):
            for j in range(-1, 2):
                if i == 0 and j == 0:
                    continue
                total += self.board[(row + i + self.rows) % self.rows, (col + j + self.cols) % self.cols]
        return total

    def update(self):
        new_board = np.copy(self.board)
        neighbor_counts_current = []  # To store neighbor counts for this iteration
        for row in range(self.rows):
            for col in range(self.cols):
                live_neighbors = self.count_neighbors(row, col)
                neighbor_counts_current.append(live_neighbors)
                if self.board[row, col] == 1:
                    if live_neighbors not in self.survival_rules:
                        new_board[row, col] = 0
                else:
                    if live_neighbors in self.birth_rules:
                        new_board[row, col] = 1
        self.board = new_board
        self.neighbor_counts.append(neighbor_counts_current)  # Add to history

    def population_count(self):
        return np.sum(self.board)

    def population_change(self):
        current_population = self.population_count()
        change = current_population - self.previous_population
        self.previous_population = current_population
        return change

    def entropy(self):
        flat_board = self.board.flatten()
        p_live = np.mean(flat_board)  # Probability of live cell
        p_dead = 1 - p_live  # Probability of dead cell

        if p_live == 0 or p_dead == 0:  # Avoid log(0)
            return 0
        return -(p_live * log2(p_live) + p_dead * log2(p_dead))

    def display(self):
        os.system('cls' if os.name == 'nt' else 'clear')

        print("┌" + "─" * (2 * self.cols) + "┐")
        for row in range(self.rows):
            print("│", end=" ")
            for col in range(self.cols):
                if self.board[row, col] == 1:
                    print(Fore.CYAN + 'O', end=" ")  # Green for live cells
                else:
                    print(Fore.BLACK + 'x', end=" ")  # Red for dead cells
            print("│")
        print("└" + "─" * (2 * self.cols) + "┘")

        # Print the metrics
        print(f"Population: {self.population_count()} | Population Change: {self.population_change()} | Entropy: {self.entropy():.4f}")

    def plot_neighbor_count_frequency(self):
        # Flatten all neighbor counts into one list
        all_neighbor_counts = [count for iteration in self.neighbor_counts for count in iteration]

        # Plot the frequency of neighbor counts
        plt.hist(all_neighbor_counts, bins=range(10), color='blue', alpha=0.7, rwidth=0.85)
        plt.title('Neighbor Count Frequency After Termination')
        plt.xlabel('Number of Neighbors')
        plt.ylabel('Frequency')
        plt.xticks(range(9))  # Neighbors can range from 0 to 8
        plt.savefig('neighbor_count_frequency.png')  # Save to filesystem
        plt.show()

    def plot_neighbor_evolution(self):
    # Create a 2D list to store frequency of neighbor counts (0 to 8) across iterations
        neighbor_frequencies = np.zeros((len(self.neighbor_counts), 9))

    # Count how many cells have each number of neighbors (0 to 8) in each iteration
        for i, iteration_counts in enumerate(self.neighbor_counts):
            for neighbors in range(9):
                neighbor_frequencies[i, neighbors] = iteration_counts.count(neighbors)

    # Plot the frequency evolution of each neighbor count
        fig, ax = plt.subplots(figsize=(10, 6))
        for neighbors in range(9):
            ax.plot(range(len(self.neighbor_counts)), neighbor_frequencies[:, neighbors], label=f'{neighbors} neighbors')

        plt.title('Neighbor Count Frequency Evolution Over Iterations')
        plt.xlabel('Iterations')
        plt.ylabel('Frequency of Cells')
        plt.legend(loc='upper right')
        plt.savefig('neighbor_evolution.png')  # Save to filesystem
        plt.show()

    def plot_population_metrics(self):
    # Create a figure with 3 subplots sharing the x-axis
        fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

        iterations = range(len(self.populations))

    # Plot Population
        axs[0].plot(iterations, self.populations, label='Population', color='green')
        axs[0].set_title('Population Over Iterations')
        axs[0].set_ylabel('Population')
        axs[0].yaxis.set_major_locator(plt.MaxNLocator(10))  # Smaller intervals

    # Plot Population Change
        axs[1].plot(iterations, self.population_changes, label='Population Change', color='blue')
        axs[1].set_title('Population Change Over Iterations')
        axs[1].set_ylabel('Population Change')
        axs[1].yaxis.set_major_locator(plt.MaxNLocator(10))  # Smaller intervals

    # Plot Entropy
        axs[2].plot(iterations, self.entropies, label='Entropy', color='red')
        axs[2].set_title('Entropy Over Iterations')
        axs[2].set_xlabel('Iterations')
        axs[2].set_ylabel('Entropy')
        axs[2].yaxis.set_major_locator(plt.MaxNLocator(10))  # Smaller intervals

    # Adjust layout and save the plot
        plt.tight_layout()
        plt.savefig('population_metrics.png')  # Save to filesystem
        plt.show()



    def run(self, iterations=1000, delay=0):
        # Variables to track previous iteration's state
        previous_population = None
        previous_population_change = None
        previous_entropy = None

        for i in range(iterations):
            # Capture current metrics
            current_population = self.population_count()
            current_population_change = self.population_change()
            current_entropy = self.entropy()

            # Track metrics for plotting
            self.populations.append(current_population)
            self.population_changes.append(current_population_change)
            self.entropies.append(current_entropy)

            # Display and update
            self.display()

            # Check for stagnation: if current values are equal to previous values and can match the next prediction, terminate early
            if (previous_population == current_population and
                previous_population_change == current_population_change and
                previous_entropy == current_entropy):
                print(f"\nReached a stagnant state. Terminating simulation early at iteration {i}.")
                break

            # Update the board for the next iteration
            self.update()
            time.sleep(delay)

            # Update previous iteration values for comparison
            previous_population = current_population
            previous_population_change = current_population_change
            previous_entropy = current_entropy

        # Display final state without clearing screen
        print(f"\nFinal Population: {self.population_count()} | Population Change: {self.population_change()} | Entropy: {self.entropy():.4f}")

        # Plot neighbor count frequency and save it
        self.plot_neighbor_count_frequency()

        self.plot_neighbor_evolution()

        # Plot population, population change, and entropy over iterations and save it
        self.plot_population_metrics()

if __name__ == "__main__":
    rows, cols = 40, 50
    birth_rules = [3]  # change these values
    survival_rules = [1, 0]

    game = GameOfLife(rows, cols, birth_rules, survival_rules)
    game.run(iterations=1000, delay=0)
